In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sys

In [4]:
#pip install pulp

In [5]:
#pip install glpk

In [6]:
#pip install cbcpy

In [7]:
#pip install glpk-utils

In [8]:
from pulp import *

In [9]:
prob = LpProblem("Simple Diet Problem",LpMinimize)

In [10]:
df = pd.read_excel("Diet Data.xlsx")
df['Price/Serving']=df['Price/Serving'].str.replace("$","")
df['Price/Serving']=df['Price/Serving'].astype('float')

# Create a list of the food items
food_items = list(df['Foods'])

# Create a dictinary of costs for all food items
costs = dict(zip(food_items,df['Price/Serving']))

# Create a dictionary of calories for all food items
calories = dict(zip(food_items,df['Calories']))

# Create a dictionary of total fat for all food items
fat = dict(zip(food_items,df['Total_Fat (g)']))

# Create a dictionary of carbohydrates for all food items
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

In [104]:
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')

In [108]:
prob = LpProblem("MAMO_Solver",LpMinimize)

In [109]:
prob

MAMO_Solver:
MINIMIZE
None
VARIABLES

In [110]:
[costs[i]*food_vars[i] for i in food_items]

[0.48*Food_Frozen_Broccoli + 0.0,
 0.54*Food_Frozen_Corn + 0.0,
 0.06*Food_Raw_Lettuce_Iceberg + 0.0,
 0.18*Food_Baked_Potatoes + 0.0,
 0.93*Food_Tofu + 0.0,
 2.52*Food_Roasted_Chicken + 0.0,
 2.34*Food_Spaghetti_W__Sauce + 0.0,
 0.72*Food_Raw_Apple + 0.0,
 0.45*Food_Banana + 0.0,
 0.15*Food_Wheat_Bread + 0.0,
 0.18*Food_White_Bread + 0.0,
 0.27*Food_Oatmeal_Cookies + 0.0,
 0.48*Food_Apple_Pie + 0.0,
 0.33*Food_Scrambled_Eggs + 0.0,
 0.45*Food_Turkey_Bologna + 0.0,
 0.81*Food_Beef_Frankfurter + 0.0,
 0.09*Food_Chocolate_Chip_Cookies + 0.0]

In [112]:
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

In [113]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0

In [118]:
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0 #"FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0 #"FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0 # "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0 #"CarbsMaximum"


In [119]:
prob.solve()

1

In [120]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])


Status: Optimal


In [122]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)
obj = value(prob.objective)
print("The total cost of this balanced diet is: ${}".format(round(obj,2)))

Food_Baked_Potatoes = 2.6953037
Food_Chocolate_Chip_Cookies = 4.3246532
The total cost of this balanced diet is: $0.87
